## Grab SafeGraph Social Distancing/Mobility data from AWS 

In [1]:
from bs4 import BeautifulSoup as bs
from bs4 import Tag, NavigableString
import re
import requests as r
import pandas as pd
import numpy as np
from datetime import datetime
import csv
import os
import sys
from state_cleaner import *
from selenium import webdriver
import matplotlib.pyplot as plt
import math
import statistics
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
os.chdir('../..')
os.chdir('CovidMobile')
os.getcwd()

'C:\\Users\\Noah\\CovidMobile'

In [3]:
#Configure the aws profile **check the slack for new access keys weekly!
#! aws configure --profile safegraph

In [4]:
#Downloads all current available data social distancing data: *takes a while to run
#! aws s3 sync s3://sg-c19-response/social-distancing/v2/ ./SafeGraph_data --profile safegraph
# aws s3 sync s3://sg-c19-response/social-distancing/v2/ ./SafeGraph_data --profile safegraph --exclude "*" --include "*2020-05-*"

In [5]:
full_df = pd.read_csv('data/Global_Mobility_Report.csv')

C:\Users\Noah\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
print(full_df['date'].min(), full_df['date'].max())

2020-02-15 2020-05-21


In [7]:
#Create list of dates to loop through
dates_obj = list(pd.date_range(start=full_df['date'].min(),end='2020-05-25'))
dates_slashes = [x.strftime("%Y/%m/%d") for x in dates_obj]
dates_dashes = [x.strftime("%Y-%m-%d") for x in dates_obj]

In [8]:
sdm_df = pd.DataFrame()

In [9]:
#Read in all of the csvs for each date:
for date1, date2 in zip(dates_slashes, dates_dashes):
    df = pd.read_csv("SafeGraph_data/{}/{}-social-distancing.csv.gz".format(date1,date2),converters={'origin_census_block_group': str})
    df = df[["origin_census_block_group", "date_range_start", "date_range_end", 
                                 "device_count", "completely_home_device_count", "part_time_work_behavior_devices", 
                                 "full_time_work_behavior_devices"]]
    df['FIPS'] = df.origin_census_block_group.str.slice(start=0, stop=5) # county is the first 5 digits of the CBG
    df['date'] = df.date_range_start.str.slice(start=0, stop=10)
    
    sdm_columns = ['device_count', 'completely_home_device_count','part_time_work_behavior_devices', 'full_time_work_behavior_devices']
    df_by_county = df.groupby(['FIPS', 'date'])[sdm_columns].sum().reset_index()
    
    sdm_df = sdm_df.append(df_by_county)
    print(date1)

2020/02/15
2020/02/16
2020/02/17
2020/02/18
2020/02/19
2020/02/20
2020/02/21
2020/02/22
2020/02/23
2020/02/24
2020/02/25
2020/02/26
2020/02/27
2020/02/28
2020/02/29
2020/03/01
2020/03/02
2020/03/03
2020/03/04
2020/03/05
2020/03/06
2020/03/07
2020/03/08
2020/03/09
2020/03/10
2020/03/11
2020/03/12
2020/03/13
2020/03/14
2020/03/15
2020/03/16
2020/03/17
2020/03/18
2020/03/19
2020/03/20
2020/03/21
2020/03/22
2020/03/23
2020/03/24
2020/03/25
2020/03/26
2020/03/27
2020/03/28
2020/03/29
2020/03/30
2020/03/31
2020/04/01
2020/04/02
2020/04/03
2020/04/04
2020/04/05
2020/04/06
2020/04/07
2020/04/08
2020/04/09
2020/04/10
2020/04/11
2020/04/12
2020/04/13
2020/04/14
2020/04/15
2020/04/16
2020/04/17
2020/04/18
2020/04/19
2020/04/20
2020/04/21
2020/04/22
2020/04/23
2020/04/24
2020/04/25
2020/04/26
2020/04/27
2020/04/28
2020/04/29
2020/04/30
2020/05/01
2020/05/02
2020/05/03
2020/05/04
2020/05/05
2020/05/06
2020/05/07
2020/05/08
2020/05/09
2020/05/10
2020/05/11
2020/05/12
2020/05/13
2020/05/14
2020/05/15

In [12]:
# convert numerical columns
sdm_df= sdm_df.astype({'device_count':'int','completely_home_device_count':'int'})

In [13]:
# compute new metrics
sdm_df['leaving_home'] = sdm_df['device_count'] - sdm_df['completely_home_device_count']
sdm_df['pct_leaving_home'] = sdm_df['leaving_home'] / sdm_df['device_count'] 

sdm_df['pct_social_distancing'] = sdm_df['completely_home_device_count']/(sdm_df['device_count']-sdm_df['full_time_work_behavior_devices']-sdm_df['part_time_work_behavior_devices'])
  
sdm_df.head()

,FIPS,date,device_count,completely_home_device_count,part_time_work_behavior_devices,full_time_work_behavior_devices,leaving_home,pct_leaving_home,pct_social_distancing
0,01001,2020-02-15,5708,1152,622,180,4556,0.798178,0.234815
1,01003,2020-02-15,23843,5359,2408,806,18484,0.775238,0.259780
2,01005,2020-02-15,1761,415,194,68,1346,0.764338,0.276851
3,01007,2020-02-15,2156,450,235,55,1706,0.791280,0.241158
4,01009,2020-02-15,6555,1242,727,251,5313,0.810526,0.222700


In [14]:
sdm_df = sdm_df.sort_values(by=['FIPS','date'])
sdm_df = sdm_df.astype({'FIPS':'float64'})

## Get SafeGraph Census data, using their Google Drive folder

Data kept in SafeGraph_data folder in GitHub, it is in .gitignore

Information about each csv can be found here: https://www.safegraph.com/blog/beginners-guide-to-census



In [15]:
counties = pd.read_csv('SafeGraph_data/safegraph_open_census_data/metadata/cbg_fips_codes.csv',
               converters={'state_fips': lambda x: str(x),'county_fips': lambda x: str(x) })
counties['FIPS'] = counties['state_fips'].astype(str)+counties['county_fips'].astype(str)
counties['FIPS'] = counties['FIPS'].astype(float)
counties = counties[['FIPS','county','state']]

In [16]:
counties.head()

,FIPS,county,state
0,1001.0,Autauga County,AL
1,1003.0,Baldwin County,AL
2,1005.0,Barbour County,AL
3,1007.0,Bibb County,AL
4,1009.0,Blount County,AL


Latitudes and longitudes are simply averaged given that the distance between them is small. To compare this method to another conversion method, see code at the bottom of the notebook.

In [17]:
codes = pd.read_csv('SafeGraph_data/safegraph_open_census_data/metadata/cbg_geographic_data.csv',
                   converters={'census_block_group': lambda x: str(x)})
codes= codes.astype({'census_block_group':'str'})
codes['FIPS'] = codes.census_block_group.str.slice(start=0, stop=5)
codes= codes.astype({'FIPS':'str'})

codes = (codes.groupby('FIPS', as_index=False)
         .agg({'latitude':'mean', 'longitude':'mean','amount_land':'sum'}))
codes['amount_land'] = codes['amount_land']/1e6 #convert to square km from square m
codes['FIPS'] =  codes['FIPS'].astype(float)

In [18]:
codes.head()

,FIPS,latitude,longitude,amount_land
0,1001.0,32.501547,-86.515629,1539.609015
1,1003.0,30.552078,-87.754456,4117.584019
2,1005.0,31.855184,-85.317696,2291.818680
3,1007.0,33.017042,-87.139093,1612.481559
4,1009.0,33.973449,-86.574490,1670.041875


In [19]:
#this keeps in two counties which don't have state/county names in the dataset
counties_and_location = counties.merge(codes, on = 'FIPS',how = 'right') 
counties_and_location.head()

,FIPS,county,state,latitude,longitude,amount_land
0,1001.0,Autauga County,AL,32.501547,-86.515629,1539.609015
1,1003.0,Baldwin County,AL,30.552078,-87.754456,4117.584019
2,1005.0,Barbour County,AL,31.855184,-85.317696,2291.818680
3,1007.0,Bibb County,AL,33.017042,-87.139093,1612.481559
4,1009.0,Blount County,AL,33.973449,-86.574490,1670.041875


In [20]:
counties_and_location[counties_and_location['state'].isnull()]

,FIPS,county,state,latitude,longitude,amount_land
3218,2158.0,NaN,NaN,62.238849,-163.313760,44240.523937
3219,46102.0,NaN,NaN,43.210922,-102.525551,5422.847813


In [21]:
def read_cbg_file(filename):
    ###Reads in specific SafeGraph file name and groups data by county FIP
    df = pd.read_csv('SafeGraph_data/safegraph_open_census_data/data/{}'.format(filename),
                     converters={'census_block_group': lambda x: str(x)})
    df = df.astype({'census_block_group':'str'})
    df['FIPS'] = df.census_block_group.str.slice(start=0, stop=5)
    df = df.groupby(['FIPS']).sum().reset_index()
    
    return df

In [22]:
sg_pop = read_cbg_file('cbg_b01.csv')

In [23]:
sg_income = read_cbg_file('cbg_b19.csv')

In [24]:
sg_race = read_cbg_file('cbg_b02.csv')

In [25]:
sg_emp_status = read_cbg_file('cbg_b23.csv')

In [26]:
sg_emp_type = read_cbg_file('cbg_c24.csv')

Demographics we want to pull and corresponding table id:
- Total Population: B01001e1
- Median Household Income: B19013e1
- Per Capita Income: B19301e1
- percent elderly (65+): 
  - Total population: B01001e1
  - Population of males 65+ is the sum of B01001e20, B01001e21, B01001e22, B01001e23, B01001e24, B01001e25
  - Population of females 65+ is the sum of B01001e44, B01001e45, B01001e46, B01001e47, B01001e48, B01001e49
  - Percent elderly  = (pop of males, females 65+)/total population

In [27]:
#Create demographics, add population and percent elderly:
dgs = sg_pop[['FIPS','B01001e1']]
dgs = dgs.rename(columns = {'B01001e1':'population'})
dgs['ln_population'] = np.log(dgs['population'])
dgs['perc_elderly'] = (sg_pop['B01001e20']+sg_pop['B01001e21']+sg_pop['B01001e22']
                           +sg_pop['B01001e23']+sg_pop['B01001e24']+ sg_pop['B01001e25']
                           +sg_pop['B01001e44']+sg_pop['B01001e45']+sg_pop['B01001e46']
                           +sg_pop['B01001e47']+sg_pop['B01001e48']+sg_pop['B01001e49'])/dgs['population']

In [28]:
dgs.head()

,FIPS,population,ln_population,perc_elderly
0,01001,55049,10.915979,0.139785
1,01003,199510,12.203620,0.187149
2,01005,26614,10.189193,0.165289
3,01007,22572,10.024465,0.148857
4,01009,57704,10.963082,0.171929


In [29]:
#Income metrics:
dgs['med_hh_income'] = sg_income['B19013e1']
dgs['per_cap_income'] = sg_income['B19301e1']

Race demographics (percentages of each race category): percentage groups less than 1% are moved to 'Other'
 - Total race population: B02001e1
 - White: B02001e2
 - Black or African American: B02001e3
 - Asian American: B02001e5
 - Native American or Alaskan Indian: B02001e4
 - Other =  the remaining percentage 

In [30]:
#Race demographics:
dgs['perc_white'] = sg_race['B02001e2']/sg_race['B02001e1']
dgs['perc_african_american'] = sg_race['B02001e3']/sg_race['B02001e1']
dgs['perc_asian'] = sg_race['B02001e5']/sg_race['B02001e1']
dgs['perc_native_american'] = sg_race['B02001e4']/sg_race['B02001e1']
dgs['perc_other_race'] = 1-dgs['perc_white']-dgs['perc_african_american']-dgs['perc_asian']-dgs['perc_native_american']

Employment Status: 
- Total pop 16+: B23025e1
- Employed: B23025e4
- Unemployed:B23025e5

In [31]:
dgs['perc_employed'] = sg_emp_status['B23025e4']/sg_emp_status['B23025e1']
dgs['perc_unemployed'] = sg_emp_status['B23025e5']/sg_emp_status['B23025e1']

Employment Type:
- Total pop, 16+: B24080e1
- Self Employed in not own incorporated business: B24080e10 + B24080e20 
- Unpaid family workers: B24080e11 + B24080e21
- Private for-profit salary workers: 
  - Employee of Private company: B24080e14 + B24080e4
  - Self-employed in own incorporated business: B24080e15 + B24080e5
- Private non-profit workers: B24080e16 + B24080e6
- Government workers (local, state, federal): B24080e17 + B24080e18 + B24080e19 + B24080e7 + B24080e8 + B24080e9

In [32]:
dgs['perc_self_employed_not_own'] = (sg_emp_type['B24080e10']+sg_emp_type['B24080e20'])/sg_emp_type['B24080e1']
dgs['perc_self_employed_own'] = (sg_emp_type['B24080e15']+sg_emp_type['B24080e5'])/sg_emp_type['B24080e1']
dgs['perc_unpaid_fam_workers'] = (sg_emp_type['B24080e11']+sg_emp_type['B24080e21'])/sg_emp_type['B24080e1']
dgs['perc_private_employees'] = (sg_emp_type['B24080e14']+sg_emp_type['B24080e4'])/sg_emp_type['B24080e1']
dgs['perc_private_non_profit'] = (sg_emp_type['B24080e16']+sg_emp_type['B24080e6'])/sg_emp_type['B24080e1']
dgs['perc_govt_workers'] = (sg_emp_type['B24080e17']+sg_emp_type['B24080e18']++sg_emp_type['B24080e19']
                           +sg_emp_type['B24080e7']+sg_emp_type['B24080e8']+sg_emp_type['B24080e9'])/sg_emp_type['B24080e1']

In [33]:
#Merge with longitudes and latitudes:
dgs['FIPS'] = dgs['FIPS'].astype(float) 
full_dgs = counties_and_location.merge(dgs, on = 'FIPS')
full_dgs.head()

,FIPS,county,state,latitude,longitude,amount_land,population,ln_population,perc_elderly,med_hh_income,...,perc_native_american,perc_other_race,perc_employed,perc_unemployed,perc_self_employed_not_own,perc_self_employed_own,perc_unpaid_fam_workers,perc_private_employees,perc_private_non_profit,perc_govt_workers
0,1001.0,Autauga County,AL,32.501547,-86.515629,1539.609015,55049,10.915979,0.139785,1578457.0,...,0.004105,0.034006,0.568037,0.033644,0.056137,0.023370,0.000000,0.640178,0.068296,0.212019
1,1003.0,Baldwin County,AL,30.552078,-87.754456,4117.584019,199510,12.203620,0.187149,4571570.0,...,0.006792,0.028981,0.547426,0.036725,0.058665,0.052215,0.001857,0.695304,0.065730,0.126229
2,1005.0,Barbour County,AL,31.855184,-85.317696,2291.818680,26614,10.189193,0.165289,777500.0,...,0.001728,0.047080,0.418747,0.061604,0.069498,0.032470,0.002002,0.662404,0.042366,0.191260
3,1007.0,Bibb County,AL,33.017042,-87.139093,1612.481559,22572,10.024465,0.148857,632309.0,...,0.003589,0.014133,0.451665,0.034764,0.059373,0.006703,0.003352,0.703974,0.056021,0.170577
4,1009.0,Blount County,AL,33.973449,-86.574490,1670.041875,57704,10.963082,0.171929,1656471.0,...,0.003483,0.024834,0.469342,0.029713,0.036586,0.030936,0.004955,0.742741,0.059742,0.125041


In [36]:
sdm_df_all = sdm_df.merge(full_dgs, on =['FIPS'])

In [37]:
sdm_df_all.to_csv('data/safegraph_data_{}.csv'.format(str(datetime.now().date()))) 

### Checking longitude and latitude average calculations with conversion to cartesian coords:

*If you want to run this, take out the group_by line bringing in the codes df

This runs for San Bernardino County, the county with the largest area in the US. The latitudes and longitudes are the same to three digits using the conversion way and also doing averages, so the averaging method was used.

In [35]:
coords_df = pd.DataFrame()
coords_df['latitude'] = codes['latitude'][codes['FIPS']=='06071']
coords_df['longitude'] = codes['longitude'][codes['FIPS']=='06071']

lats = list(codes['latitude'][codes['FIPS']=='06071'])
lons = list(codes['longitude'][codes['FIPS']=='06071'])

In [36]:
x = 0.0
y = 0.0
z = 0.0

for i, coord in coords_df.iterrows():
    latitude = math.radians(coord.latitude)
    longitude = math.radians(coord.longitude)

    x += math.cos(latitude) * math.cos(longitude)
    y += math.cos(latitude) * math.sin(longitude)
    z += math.sin(latitude)

total = len(coords_df)

x = x / total
y = y / total
z = z / total

central_longitude = math.atan2(y, x)
central_square_root = math.sqrt(x * x + y * y)
central_latitude = math.atan2(z, central_square_root)

mean_location = {
    'latitude': math.degrees(central_latitude),
    'longitude': math.degrees(central_longitude)
    }

In [37]:
mean_location

{'latitude': 34.18227596007321, 'longitude': -117.32819489532969}

In [38]:
(sum(lats)/len(lats), sum(lons)/len(lons))

(34.18227596007321, -117.32819489532969)